In [1]:
from sklearn.metrics import accuracy_score#(y_true, y_pred,
from pyriemann.tangentspace import TangentSpace
from pyriemann.estimation import Covariances
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mne.filter import filter_data
import pandas as pd
from mrmr import mrmr_classif

import mne
mne.set_log_level('ERROR')

# Load the data

In [2]:
from scripts.scripts import *
dict_config = { 
'model':'EEGSimpleConv',
'params':[84,1,95,13],
'dataset':'BNCI',
'EA':False,
'EOG':False,
'Z':'Z0',
'path':'/users/local/simpleconv_datasets',
'lmso':False,
'session':True,
'reg_subject':False,
'use_wandb':False,
'evaluation':'within',
'within':True,
'mdl':False,
'filter':0.5,
}  

    
    
X,Y = load_data(dict_config)

Sampling Freq: 250


# CSP

In [3]:
acc_model = []
for s in range(len(X)):
    data_train = X[s][0].numpy().astype(np.double)
    y_train = Y[s][0].numpy() # labels
    data_test  = X[s][1].numpy().astype(np.double)
    y_test  = Y[s][1].numpy()
    data_train = mne.filter.filter_data(data_train, 250, 4, 40)
    data_test = mne.filter.filter_data(data_test, 250, 4, 40)
    csp = CSP(n_components=12)
    data_train = csp.fit_transform(data_train.astype(np.double),y_train.astype(np.double))
    data_test = csp.transform(data_test)
    model = LinearDiscriminantAnalysis(shrinkage='auto', solver='lsqr')
    model.fit(data_train,y_train)
    y_pred = model.predict(data_test)
    acc = accuracy_score(y_pred,y_test)
    acc_model.append(acc)
    print('S'+str(s)+':',round(acc*100,2))

S0: 68.06
S1: 52.78
S2: 73.61
S3: 53.12
S4: 27.78
S5: 42.36
S6: 55.56
S7: 71.53
S8: 74.31


In [4]:
print('Avg accuracy:',round(np.array(acc_model).mean()*100,2),'±',round(np.array(acc_model).std()*100,2))

Avg accuracy: 57.68 ± 14.94


# FBCSP

In [5]:
def filter_bank(data,sfreq):
    bank = []
    for i in np.arange(4,40,4):
        bank.append(mne.filter.filter_data(data, sfreq, i, i+4,method='iir'))
    return bank

def csp_bank(data_train,labels_train,data_test):
    csp = CSP(n_components=4)
    csp_train = csp.fit_transform(data_train,labels_train)
    csp_test = csp.transform(data_test)
    return csp_train,csp_test

In [6]:
train_bank = []
test_bank = []
for i in range(9):
    train_bank.append(filter_bank(X[i][0].numpy().astype(np.double),250))
    test_bank.append(filter_bank(X[i][1].numpy().astype(np.double),250))

In [7]:
list_acc_fbcsp = []

for i in range(9):
    eeg_y_train = Y[i][0].numpy().astype(np.double)
    eeg_y_test = Y[i][1].numpy().astype(np.double)
    fbank_train = train_bank[i]
    fbank_test = test_bank[i]
    cspbank_train_ = []
    cspbank_test_ = []
    for b_tr,b_te in zip(fbank_train,fbank_test):
        tr,te = csp_bank(b_tr,eeg_y_train,b_te)
        cspbank_train_.append(tr)
        cspbank_test_.append(te)
        
    cspbank_train = np.concatenate(cspbank_train_,1)
    cspbank_test = np.concatenate(cspbank_test_,1)
    
    selected_features = mrmr_classif(X=pd.DataFrame(cspbank_train), y=eeg_y_train, K=18)
    model = LinearDiscriminantAnalysis(shrinkage='auto', solver='lsqr')
    x_train =cspbank_train[:,selected_features]
    x_test =cspbank_test[:,selected_features]
    model.fit(x_train,eeg_y_train)
    y_pred = model.predict(x_test)
    acc = accuracy_score(y_pred,eeg_y_test)
    print('S'+str(i)+':',round(acc*100,2))
    list_acc_fbcsp.append(acc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  4.69it/s]


S0: 75.35


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:04<00:00,  4.36it/s]


S1: 56.25


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  5.13it/s]


...

S6: 74.31


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  4.99it/s]


S7: 63.19


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  4.90it/s]

S8: 63.19


In [8]:
print('Avg accuracy:',round(np.array(list_acc_fbcsp).mean()*100,2),'±',round(np.array(list_acc_fbcsp).std()*100,2))

Avg accuracy: 63.73 ± 10.4


# TS+LDA

In [9]:
cov = Covariances(estimator ='oas')
acc_model = []
for s in range(len(X)):
    X_train = X[s][0].numpy()
    y_train = Y[s][0].numpy() # labels
    X_test  = X[s][1].numpy()
    y_test  = Y[s][1].numpy()
    X_train = filter_data(X_train.astype(float),250,7,35)
    X_test = filter_data(X_test.astype(float),250,7,35)
    cov_train = cov.fit_transform(X_train)
    cov_test = cov.transform(X_test)
    ts = TangentSpace()
    train_ts = ts.fit_transform(cov_train)
    test_ts = ts.transform(cov_test)
    model = LinearDiscriminantAnalysis(shrinkage='auto', solver='eigen')
    model.fit(train_ts,y_train)
    y_pred = model.predict(test_ts)
    acc = accuracy_score(y_pred,y_test)
    print('S'+str(s)+':',round(acc*100,2))
    acc_model.append(acc)

S0: 77.43
S1: 52.78
S2: 84.03
S3: 60.07
S4: 47.92
S5: 49.31
S6: 64.24
S7: 73.96
S8: 78.82


In [10]:
print('Avg accuracy:',round(np.array(acc_model).mean()*100,2),'±',round(np.array(acc_model).std()*100,2))

Avg accuracy: 65.39 ± 12.9
